# Recommender system in PyTorch

PyTorch is basically a deep learning framework. This means that we can solve a huge amount of real-world problems using pyTorch.

Even before the re-rise of deep learning approaches, the field of machine learning and data analysis were beloved research topics of this field. These approaches were not just studied in an academy. There were a lot of applications, and among them, recommender system is a representative one that made a great success in real-world.

Recommender system is a model that predicts which item a user will like. As a prior knowledge, we will be given a data on user's preference on some of the items that the user has already rated. Additional information on users or items could be given as an extra data. There are some different types of models, but the most popular form of the model is the one that predicts user's preferences on items that the user has not rated yet. In other words, it is to predict the missing data from the known data. After then, for a specific user, we can sort the items that the user has not rated by their predicted preference in decreasing order, and suggest the first few items to the user. There has been a lot of methods to make such predictions, including a lot of ones that does not rely on deep learning.

In this tutorial, we'll take a look at the classical recommender system. Perhaps the *Netflix Prize* is a competition that took researchers' interest. It was a competition to predict users' star-rating on the movies with some of the given rating data. We'll follow the similar way with the similar data. Although it was not the final winner of the competition, but we'll take a deeper look on a model that made a historical, and the most important breakthrough in this field; It is a collaborative filtering method which uses a singular value decomposition.

Please keep in mind that  neural networks will not appear in this tutorial. Instead, please enjoy the way how a deep learning framework is used on a work without deep learning!

## 1. Introduction

### The Problem

First of all, let's define the problem. Suppose that you're maintaing a website where users rate the movies. There are $N$ users and $M$ movies. Some of the users have already rated some movies. You'll be given the rating data as a input of the problem. For the output, you have to predict the rest of the rating data; How the users will rate the movies which they haven't rated yet.

How would you solve this problem? If you have no idea, then just think about the movies you like. Suppose that you like a movie with particular acting person, or a movie directed by a particular director, or a movie of a specific genre. Then you'll probably like the ones with the same acting person, the same director, or the same genre. Or suppose that you have a friend who has the same list of favorite movies. If there's a new movie that your friend likes, then you'll probably like that as well. These are well known approach to this problem, and of course, there are tons of other ideas.

But how it is related with a recommendation? If you can predict a user's rating on movies, then you can also predict the list of the movies the user will like. Predict the rating, sort them, and just print the first few movies in the list which have ratings over a pre-determined threshold! So predicting the rating is simply everything of a recommender system.